# Working with HEALPix Data

In this notebook, we showcase how UXarray can be used with HEALPix data.
* A brief overview into HEALPix
* Representing a HEALPix grid in the UGRID conventions
* Loading data that resides on HEALPix Grids

In [ ]:
import uxarray as ux
import cartopy.crs as ccrs

## What is HEALPix?

The Hierarchical Equal Area isoLatitude Pixelisation (HEALPix) algorithm is a method for the pixelisation of the
2-sphere. It has three defining qualities.
- The sphere is hierarchically tessellated into curvilinear quadrilaterals
- Areas of all pixels at a given resolution are identical
- Pixels are distributed on lines of constant latitude

```{note}
For more information on HEALPix, you can refer to [this excellent overview](https://easy.gems.dkrz.de/Processing/healpix/index.html#hierarchical-healpix-output) in DKRZ's Easy Gems documentation.
```






## Representing a HEALPix Grid in the UGRID Conventions



The number of cells in a HEALPix grid depends on the choice of `zoom` level, following the equation $12 \cdot z^4$.

The ``ux.Grid.from_healpix()`` classmethod can be used to represent a HEALPix Grid in the UGRID conventions. Given a zoom level, it will construct a ``ux.Grid`` instance representing a HEALPix grid.








In [ ]:
ux.Grid.from_healpix(zoom=3)

Since much of UXarray's functionality requires explicit defined connectivity arrays. The most important one is the `face_node_connectivity`, which contains the indices of the boundaries of each pixel. This also constructs the `node_lat` and `node_lon` variables, corresponding to the latitude and longitude of the boundaries of each pixel.

In [ ]:
ux.Grid.from_healpix(zoom=3, pixels_only=False)

However, even if you load in a HEALPix grid specifying that you do not want the connectivity upfront, they can still be constructed when desired because of UXarray's internal design.

In [ ]:
ux.Grid.from_healpix(zoom=3, pixels_only=True).face_node_connectivity

### Plotting

Once loaded in, UXarray does not care about the underlying grid format as it represents all formats in its unified UGRID-like convention; hence, all existing functionality is supported on HEALPix through the same UXarray interface, such as plotting.

In [ ]:
(
    ux.Grid.from_healpix(zoom=2).plot(
        periodic_elements="split", projection=ccrs.Orthographic(), title="zoom=2"
    )
    + ux.Grid.from_healpix(zoom=3).plot(
        periodic_elements="split", projection=ccrs.Orthographic(), title="zoom=3"
    )
    + ux.Grid.from_healpix(zoom=4).plot(
        periodic_elements="split", projection=ccrs.Orthographic(), title="zoom=4"
    )
).cols(1)

### Remapping

If you are looking to remap data from one grid format to HEALPix, UXarray's remapping functionality can be used.

In [ ]:
source_uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)

source_uxds["psi"].plot(
    cmap="inferno", projection=ccrs.Orthographic(), title="Source Data"
)

In [ ]:
hp_grid = ux.Grid.from_healpix(zoom=5)

In [ ]:
source_uxds.uxgrid.plot(
    projection=ccrs.Orthographic(), title="Source Grid (CSne30)"
) + hp_grid.plot(projection=ccrs.Orthographic(), title="Destination Grid (HEALPix)")

In [ ]:
psi_hp = source_uxds["psi"].remap.nearest_neighbor(destination_grid=hp_grid)
psi_hp

In [ ]:
psi_hp.plot(cmap="inferno", projection=ccrs.Orthographic(), title="Remapped Data")

We can chain together a few functions to convert our output from a UXarray Dataset to a NetCDF file, with the grid represented in the HEALPix format.
- `to_dataset()`: Converts the ``ux.UxDataArray`` to a ``ux.UxDataset``
- `to_xarray(grid_format="HEALPix")`: Converts our ``ux.UxDataset`` to a ``xr.Dataset`` encoded in the HEALPix format.
- `to_netcdf("psi_healpix.nc")`: Saves our dataset to disk as a NetCDF file.

In [ ]:
psi_hp.to_dataset().to_xarray(grid_format="HEALPix").to_netcdf("psi_healpix.nc")

## Loading HEALPix Data in UXarray

Using the example we generated above, we can load in data that is stored in the HEALPix format using the ``UxDataset.from_healpix()`` classmethod.

In [ ]:
uxds = ux.UxDataset.from_healpix("psi_healpix.nc")
uxds

When a HEALPix dataset is loaded, the zoom level is automatically selected based off the size of the `cell` dimension in the original dataset. By printing our grid, we can see that our attached grid has a zoom level of 5.

In [ ]:
uxds.uxgrid